In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import sys
import pyodbc as odbc

In [3]:
cnxn = odbc.connect('Driver={SQL Server};'
                    'Server=DESKTOP-S40OV4H;'
                    'Database=Sum_capstone_proj;'
                    'Trusted_Connection=yes;')

In [4]:
#Importing the dataset, read straight from SQL
dataset=pd.read_sql_query('SELECT * FROM diamonds', cnxn)
dataset.head(10)

,carat,cut,color,clarity,depth,table,x,y,z,price
0,0.23,Ideal,E,SI2,61.5,55.0,3.95,3.98,2.43,326.0
1,0.21,Premium,E,SI1,59.8,61.0,3.89,3.84,2.31,326.0
2,0.23,Good,E,VS1,56.9,65.0,4.05,4.07,2.31,327.0
3,0.29,Premium,I,VS2,62.4,58.0,4.20,4.23,2.63,334.0
4,0.31,Good,J,SI2,63.3,58.0,4.34,4.35,2.75,335.0
5,0.24,Very Good,J,VVS2,62.8,57.0,3.94,3.96,2.48,336.0
6,0.24,Very Good,I,VVS1,62.3,57.0,3.95,3.98,2.47,336.0
7,0.26,Very Good,H,SI1,61.9,55.0,4.07,4.11,2.53,337.0
8,0.22,Fair,E,VS2,65.1,61.0,3.87,3.78,2.49,337.0
9,0.23,Very Good,H,VS1,59.4,61.0,4.00,4.05,2.39,338.0


In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  object 
 2   color    53940 non-null  object 
 3   clarity  53940 non-null  object 
 4   depth    53940 non-null  float64
 5   table    53940 non-null  float64
 6   x        53940 non-null  float64
 7   y        53940 non-null  float64
 8   z        53940 non-null  float64
 9   price    53940 non-null  float64
dtypes: float64(7), object(3)
memory usage: 4.1+ MB


In [6]:
dataset.describe()
#Zero values observed in x, y, z

,carat,depth,table,x,y,z,price
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,61.749405,57.457184,5.731157,5.734526,3.538734,3932.799722
std,0.474011,1.432621,2.234491,1.121761,1.142135,0.705699,3989.439738
min,0.200000,43.000000,43.000000,0.000000,0.000000,0.000000,326.000000
25%,0.400000,61.000000,56.000000,4.710000,4.720000,2.910000,950.000000
50%,0.700000,61.800000,57.000000,5.700000,5.710000,3.530000,2401.000000
75%,1.040000,62.500000,59.000000,6.540000,6.540000,4.040000,5324.250000
max,5.010000,79.000000,95.000000,10.740000,58.900000,31.800000,18823.000000


Using SQL to check the data for zero values observed in x, y, z,
it was found that 20 rows were affected. It was decided not to remove the 20 rows of data
as the price can still be predicted from the other parameters (eg. carat, cut, etc.).
Just take note that for the Machine Learning, we will exclude the parameters x, y, z.

Feature Engineering - convert cut, color & clarity from categorical to numeric value

In [7]:
cut_mapping = {"Ideal": 5, "Premium": 4, "Very Good": 3, "Good": 2, "Fair": 1}
dataset["cut_scale"] = dataset.cut.map(cut_mapping)
dataset.head(10)

,carat,cut,color,clarity,depth,table,x,y,z,price,cut_scale
0,0.23,Ideal,E,SI2,61.5,55.0,3.95,3.98,2.43,326.0,5
1,0.21,Premium,E,SI1,59.8,61.0,3.89,3.84,2.31,326.0,4
2,0.23,Good,E,VS1,56.9,65.0,4.05,4.07,2.31,327.0,2
3,0.29,Premium,I,VS2,62.4,58.0,4.20,4.23,2.63,334.0,4
4,0.31,Good,J,SI2,63.3,58.0,4.34,4.35,2.75,335.0,2
5,0.24,Very Good,J,VVS2,62.8,57.0,3.94,3.96,2.48,336.0,3
6,0.24,Very Good,I,VVS1,62.3,57.0,3.95,3.98,2.47,336.0,3
7,0.26,Very Good,H,SI1,61.9,55.0,4.07,4.11,2.53,337.0,3
8,0.22,Fair,E,VS2,65.1,61.0,3.87,3.78,2.49,337.0,1
9,0.23,Very Good,H,VS1,59.4,61.0,4.00,4.05,2.39,338.0,3


In [8]:
color_mapping = {"D": 7, "E": 6, "F": 5, "G": 4, "H": 3, "I": 2, "J": 1}
dataset["color_scale"] = dataset.color.map(color_mapping)
dataset.head(10)

,carat,cut,color,clarity,depth,table,x,y,z,price,cut_scale,color_scale
0,0.23,Ideal,E,SI2,61.5,55.0,3.95,3.98,2.43,326.0,5,6
1,0.21,Premium,E,SI1,59.8,61.0,3.89,3.84,2.31,326.0,4,6
2,0.23,Good,E,VS1,56.9,65.0,4.05,4.07,2.31,327.0,2,6
3,0.29,Premium,I,VS2,62.4,58.0,4.20,4.23,2.63,334.0,4,2
4,0.31,Good,J,SI2,63.3,58.0,4.34,4.35,2.75,335.0,2,1
5,0.24,Very Good,J,VVS2,62.8,57.0,3.94,3.96,2.48,336.0,3,1
6,0.24,Very Good,I,VVS1,62.3,57.0,3.95,3.98,2.47,336.0,3,2
7,0.26,Very Good,H,SI1,61.9,55.0,4.07,4.11,2.53,337.0,3,3
8,0.22,Fair,E,VS2,65.1,61.0,3.87,3.78,2.49,337.0,1,6
9,0.23,Very Good,H,VS1,59.4,61.0,4.00,4.05,2.39,338.0,3,3


In [9]:
clarity_mapping = {"FL":11, "IF": 10, "VVS1": 9, "VVS2": 8, "VS1": 7, "VS2": 6, "SI1": 5, "SI2": 4, "I1": 3, "I2": 2, "I3":1}
dataset["clarity_scale"] = dataset.clarity.map(clarity_mapping)
dataset.head(20)

,carat,cut,color,clarity,depth,table,x,y,z,price,cut_scale,color_scale,clarity_scale
0,0.23,Ideal,E,SI2,61.5,55.0,3.95,3.98,2.43,326.0,5,6,4
1,0.21,Premium,E,SI1,59.8,61.0,3.89,3.84,2.31,326.0,4,6,5
2,0.23,Good,E,VS1,56.9,65.0,4.05,4.07,2.31,327.0,2,6,7
3,0.29,Premium,I,VS2,62.4,58.0,4.20,4.23,2.63,334.0,4,2,6
4,0.31,Good,J,SI2,63.3,58.0,4.34,4.35,2.75,335.0,2,1,4
5,0.24,Very Good,J,VVS2,62.8,57.0,3.94,3.96,2.48,336.0,3,1,8
6,0.24,Very Good,I,VVS1,62.3,57.0,3.95,3.98,2.47,336.0,3,2,9
7,0.26,Very Good,H,SI1,61.9,55.0,4.07,4.11,2.53,337.0,3,3,5
8,0.22,Fair,E,VS2,65.1,61.0,3.87,3.78,2.49,337.0,1,6,6
9,0.23,Very Good,H,VS1,59.4,61.0,4.00,4.05,2.39,338.0,3,3,7


Rearranging columns

In [10]:
dataset["carat_col"] = dataset.carat
dataset["price_USD"] = dataset.price
dataset['index_col'] = dataset.index
dataset.head(30)
del dataset['carat']
del dataset['price']
dataset.head(20)

,cut,color,clarity,depth,table,x,y,z,cut_scale,color_scale,clarity_scale,carat_col,price_USD,index_col
0,Ideal,E,SI2,61.5,55.0,3.95,3.98,2.43,5,6,4,0.23,326.0,0
1,Premium,E,SI1,59.8,61.0,3.89,3.84,2.31,4,6,5,0.21,326.0,1
2,Good,E,VS1,56.9,65.0,4.05,4.07,2.31,2,6,7,0.23,327.0,2
3,Premium,I,VS2,62.4,58.0,4.20,4.23,2.63,4,2,6,0.29,334.0,3
4,Good,J,SI2,63.3,58.0,4.34,4.35,2.75,2,1,4,0.31,335.0,4
5,Very Good,J,VVS2,62.8,57.0,3.94,3.96,2.48,3,1,8,0.24,336.0,5
6,Very Good,I,VVS1,62.3,57.0,3.95,3.98,2.47,3,2,9,0.24,336.0,6
7,Very Good,H,SI1,61.9,55.0,4.07,4.11,2.53,3,3,5,0.26,337.0,7
8,Fair,E,VS2,65.1,61.0,3.87,3.78,2.49,1,6,6,0.22,337.0,8
9,Very Good,H,VS1,59.4,61.0,4.00,4.05,2.39,3,3,7,0.23,338.0,9


Remove unwanted columns

In [11]:
#Remove x, y, z from table, decided not use them for predicting price
del dataset['x']
del dataset['y']
del dataset['z']
dataset.head(20)

,cut,color,clarity,depth,table,cut_scale,color_scale,clarity_scale,carat_col,price_USD,index_col
0,Ideal,E,SI2,61.5,55.0,5,6,4,0.23,326.0,0
1,Premium,E,SI1,59.8,61.0,4,6,5,0.21,326.0,1
2,Good,E,VS1,56.9,65.0,2,6,7,0.23,327.0,2
3,Premium,I,VS2,62.4,58.0,4,2,6,0.29,334.0,3
4,Good,J,SI2,63.3,58.0,2,1,4,0.31,335.0,4
5,Very Good,J,VVS2,62.8,57.0,3,1,8,0.24,336.0,5
6,Very Good,I,VVS1,62.3,57.0,3,2,9,0.24,336.0,6
7,Very Good,H,SI1,61.9,55.0,3,3,5,0.26,337.0,7
8,Fair,E,VS2,65.1,61.0,1,6,6,0.22,337.0,8
9,Very Good,H,VS1,59.4,61.0,3,3,7,0.23,338.0,9


Prep to apply scalar to data to allow better algorithm convergence

In [12]:
#Defining X and y
X=dataset.iloc[:,3:-2].values
y=dataset.iloc[:,9:11].values

In [13]:
X

array([[61.5 , 55.  ,  5.  ,  6.  ,  4.  ,  0.23],
       [59.8 , 61.  ,  4.  ,  6.  ,  5.  ,  0.21],
       [56.9 , 65.  ,  2.  ,  6.  ,  7.  ,  0.23],
       ...,
       [62.8 , 60.  ,  3.  ,  7.  ,  5.  ,  0.7 ],
       [61.  , 58.  ,  4.  ,  3.  ,  4.  ,  0.86],
       [62.2 , 55.  ,  5.  ,  7.  ,  4.  ,  0.75]])

In [14]:
y #NOT NEEDED

array([[3.2600e+02, 0.0000e+00],
       [3.2600e+02, 1.0000e+00],
       [3.2700e+02, 2.0000e+00],
       ...,
       [2.7570e+03, 5.3937e+04],
       [2.7570e+03, 5.3938e+04],
       [2.7570e+03, 5.3939e+04]])

In [15]:
X.shape

(53940, 6)

In [16]:
y.shape

(53940, 2)

Idea is that all training, test, and input for prediction data HAVE TO BE scaled with the same scalar for prediction to work.

In [17]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_scfitransform=sc.fit_transform(X)
X_scfitransform

array([[-0.17409151, -1.09967199,  0.98147332,  0.93716275, -1.24521508,
        -1.19816781],
       [-1.36073849,  1.58552871,  0.08588908,  0.93716275, -0.63809506,
        -1.24036129],
       [-3.38501862,  3.37566251, -1.70527938,  0.93716275,  0.57614496,
        -1.19816781],
       ...,
       [ 0.73334442,  1.13799526, -0.80969515,  1.52502147, -0.63809506,
        -0.20662095],
       [-0.52310533,  0.24292836,  0.08588908, -0.8264134 , -1.24521508,
         0.13092691],
       [ 0.31452784, -1.09967199,  0.98147332,  1.52502147, -1.24521508,
        -0.10113725]])

In [18]:
df=pd.DataFrame(data=X_scfitransform[0:,0:],
            index=[i for i in range(X_scfitransform.shape[0])],
            columns=['f'+str(i) for i in range(X_scfitransform.shape[1])])

In [19]:
df.head()

,f0,f1,f2,f3,f4,f5
0,-0.174092,-1.099672,0.981473,0.937163,-1.245215,-1.198168
1,-1.360738,1.585529,0.085889,0.937163,-0.638095,-1.240361
2,-3.385019,3.375663,-1.705279,0.937163,0.576145,-1.198168
3,0.454133,0.242928,0.085889,-1.414272,-0.030975,-1.071587
4,1.082358,0.242928,-1.705279,-2.002131,-1.245215,-1.029394


In [20]:
df['depth_scaled'] = df.f0
df['table_scaled'] = df.f1
df['cut_scaled'] = df.f2
df['color_scaled'] = df.f3
df['clarity_scaled'] = df.f4
df['carat_scaled'] = df.f5
df['depth'] = dataset.depth
df['table'] = dataset.table
df['cut'] = dataset.cut_scale
df['color'] = dataset.color_scale
df['clarity'] = dataset.clarity_scale
df['carat'] = dataset.carat_col
df['price'] = dataset.price_USD
df['index_no'] = df.index
del df['f0']
del df['f1']
del df['f2']
del df['f3']
del df['f4']
del df['f5']
df.head()

,depth_scaled,table_scaled,cut_scaled,color_scaled,clarity_scaled,carat_scaled,depth,table,cut,color,clarity,carat,price,index_no
0,-0.174092,-1.099672,0.981473,0.937163,-1.245215,-1.198168,61.5,55.0,5,6,4,0.23,326.0,0
1,-1.360738,1.585529,0.085889,0.937163,-0.638095,-1.240361,59.8,61.0,4,6,5,0.21,326.0,1
2,-3.385019,3.375663,-1.705279,0.937163,0.576145,-1.198168,56.9,65.0,2,6,7,0.23,327.0,2
3,0.454133,0.242928,0.085889,-1.414272,-0.030975,-1.071587,62.4,58.0,4,2,6,0.29,334.0,3
4,1.082358,0.242928,-1.705279,-2.002131,-1.245215,-1.029394,63.3,58.0,2,1,4,0.31,335.0,4


In [21]:
#Output as array then shuffle
array=df.iloc[:,:].values
array

array([[-1.74091508e-01, -1.09967199e+00,  9.81473316e-01, ...,
         2.30000000e-01,  3.26000000e+02,  0.00000000e+00],
       [-1.36073849e+00,  1.58552871e+00,  8.58890847e-02, ...,
         2.10000000e-01,  3.26000000e+02,  1.00000000e+00],
       [-3.38501862e+00,  3.37566251e+00, -1.70527938e+00, ...,
         2.30000000e-01,  3.27000000e+02,  2.00000000e+00],
       ...,
       [ 7.33344416e-01,  1.13799526e+00, -8.09695147e-01, ...,
         7.00000000e-01,  2.75700000e+03,  5.39370000e+04],
       [-5.23105325e-01,  2.42928358e-01,  8.58890847e-02, ...,
         8.60000000e-01,  2.75700000e+03,  5.39380000e+04],
       [ 3.14527836e-01, -1.09967199e+00,  9.81473316e-01, ...,
         7.50000000e-01,  2.75700000e+03,  5.39390000e+04]])

In [22]:
Q = array

In [23]:
Q

array([[-1.74091508e-01, -1.09967199e+00,  9.81473316e-01, ...,
         2.30000000e-01,  3.26000000e+02,  0.00000000e+00],
       [-1.36073849e+00,  1.58552871e+00,  8.58890847e-02, ...,
         2.10000000e-01,  3.26000000e+02,  1.00000000e+00],
       [-3.38501862e+00,  3.37566251e+00, -1.70527938e+00, ...,
         2.30000000e-01,  3.27000000e+02,  2.00000000e+00],
       ...,
       [ 7.33344416e-01,  1.13799526e+00, -8.09695147e-01, ...,
         7.00000000e-01,  2.75700000e+03,  5.39370000e+04],
       [-5.23105325e-01,  2.42928358e-01,  8.58890847e-02, ...,
         8.60000000e-01,  2.75700000e+03,  5.39380000e+04],
       [ 3.14527836e-01, -1.09967199e+00,  9.81473316e-01, ...,
         7.50000000e-01,  2.75700000e+03,  5.39390000e+04]])

In [24]:
from sklearn.utils import shuffle
Q = shuffle(array,random_state=69) #dataset gets shuffled
Q

array([[ 7.33344416e-01, -2.04605091e-01,  9.81473316e-01, ...,
         6.60000000e-01,  1.90100000e+03,  4.78030000e+04],
       [ 1.05119545e-01, -6.52138541e-01,  9.81473316e-01, ...,
         1.06000000e+00,  1.35880000e+04,  2.50680000e+04],
       [ 9.42752706e-01,  2.42928358e-01, -1.70527938e+00, ...,
         7.40000000e-01,  2.82400000e+03,  5.18000000e+02],
       ...,
       [ 2.12939968e+00, -6.52138541e-01, -2.60086361e+00, ...,
         1.50000000e+00,  1.35530000e+04,  2.50150000e+04],
       [-2.05876612e+00,  6.90461808e-01,  8.58890847e-02, ...,
         1.01000000e+00,  4.67200000e+03,  9.81800000e+03],
       [ 2.44725072e-01, -6.52138541e-01,  9.81473316e-01, ...,
         1.00000000e+00,  3.52800000e+03,  4.04100000e+03]])

In [25]:
array2df=pd.DataFrame(data=Q[0:,0:],
            index=[i for i in range(Q.shape[0])],
            columns=['f'+str(i) for i in range(Q.shape[1])])
array2df

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13
0,0.733344,-0.204605,0.981473,0.937163,-0.638095,-0.291008,62.8,57.0,5.0,6.0,5.0,0.66,1901.0,47803.0
1,0.105120,-0.652139,0.981473,1.525021,1.183265,0.552862,61.9,56.0,5.0,7.0,8.0,1.06,13588.0,25068.0
2,0.942753,0.242928,-1.705279,0.937163,0.576145,-0.122234,63.1,58.0,2.0,6.0,7.0,0.74,2824.0,518.0
3,1.570978,-1.994739,-1.705279,-1.414272,0.576145,0.426281,64.0,53.0,2.0,2.0,7.0,1.00,3763.0,5135.0
4,-0.523105,1.137995,0.085889,-0.826413,-0.638095,1.481118,61.0,60.0,4.0,3.0,5.0,1.50,9820.0,21753.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53935,0.942753,0.690462,-1.705279,0.937163,-0.638095,0.595055,63.1,59.0,2.0,6.0,5.0,1.08,5096.0,11825.0
53936,0.035317,-0.204605,0.981473,-0.238555,-1.245215,-0.164427,61.8,57.0,5.0,4.0,4.0,0.72,1866.0,47508.0
53937,2.129400,-0.652139,-2.600864,-0.238555,1.183265,1.481118,64.8,56.0,1.0,4.0,8.0,1.50,13553.0,25015.0
53938,-2.058766,0.690462,0.085889,1.525021,-1.245215,0.447378,58.8,59.0,4.0,7.0,4.0,1.01,4672.0,9818.0


In [26]:
array2df['depth_scaled'] = array2df.f0
array2df['table_scaled'] = array2df.f1
array2df['cut_scaled'] = array2df.f2
array2df['color_scaled'] = array2df.f3
array2df['clarity_scaled'] = array2df.f4
array2df['carat_scaled'] = array2df.f5
array2df['depth'] = array2df.f6
array2df['table'] = array2df.f7
array2df['cut'] = array2df.f8
array2df['color'] = array2df.f9
array2df['clarity'] = array2df.f10
array2df['carat'] = array2df.f11
array2df['price_USD'] = array2df.f12
array2df['index_org'] = array2df.f13
del array2df['f0']
del array2df['f1']
del array2df['f2']
del array2df['f3']
del array2df['f4']
del array2df['f5']
del array2df['f6']
del array2df['f7']
del array2df['f8']
del array2df['f9']
del array2df['f10']
del array2df['f11']
del array2df['f12']
del array2df['f13']
array2df.head()

,depth_scaled,table_scaled,cut_scaled,color_scaled,clarity_scaled,carat_scaled,depth,table,cut,color,clarity,carat,price_USD,index_org
0,0.733344,-0.204605,0.981473,0.937163,-0.638095,-0.291008,62.8,57.0,5.0,6.0,5.0,0.66,1901.0,47803.0
1,0.105120,-0.652139,0.981473,1.525021,1.183265,0.552862,61.9,56.0,5.0,7.0,8.0,1.06,13588.0,25068.0
2,0.942753,0.242928,-1.705279,0.937163,0.576145,-0.122234,63.1,58.0,2.0,6.0,7.0,0.74,2824.0,518.0
3,1.570978,-1.994739,-1.705279,-1.414272,0.576145,0.426281,64.0,53.0,2.0,2.0,7.0,1.00,3763.0,5135.0
4,-0.523105,1.137995,0.085889,-0.826413,-0.638095,1.481118,61.0,60.0,4.0,3.0,5.0,1.50,9820.0,21753.0


In [ ]:
#array2df.to_csv('diamonds_shuf69sclnzparm_53940all.csv',index=False)

In [ ]:
#array2df.head(50000).to_csv('diamonds_shuf69sclnzparm_50k.csv',index=False)
#array2df.head(10000).to_csv('diamonds_shuf69sclnzparm_10k.csv',index=False)
#array2df.tail(3940).to_csv('diamonds_shuf69sclnzparm_tail_3940.csv',index=False)

In [29]:
#As input into SQL cannot have too many attributes, remove no longer needed attributes here.
del array2df['depth']
del array2df['table']
del array2df['cut']
del array2df['color']
del array2df['clarity']
del array2df['carat']
array2df.head()

,depth_scaled,table_scaled,cut_scaled,color_scaled,clarity_scaled,carat_scaled,price_USD,index_org
0,0.733344,-0.204605,0.981473,0.937163,-0.638095,-0.291008,1901.0,47803.0
1,0.105120,-0.652139,0.981473,1.525021,1.183265,0.552862,13588.0,25068.0
2,0.942753,0.242928,-1.705279,0.937163,0.576145,-0.122234,2824.0,518.0
3,1.570978,-1.994739,-1.705279,-1.414272,0.576145,0.426281,3763.0,5135.0
4,-0.523105,1.137995,0.085889,-0.826413,-0.638095,1.481118,9820.0,21753.0


In [30]:
array2df_head_50k = array2df.head(50000)
array2df_head_50k

,depth_scaled,table_scaled,cut_scaled,color_scaled,clarity_scaled,carat_scaled,price_USD,index_org
0,0.733344,-0.204605,0.981473,0.937163,-0.638095,-0.291008,1901.0,47803.0
1,0.105120,-0.652139,0.981473,1.525021,1.183265,0.552862,13588.0,25068.0
2,0.942753,0.242928,-1.705279,0.937163,0.576145,-0.122234,2824.0,518.0
3,1.570978,-1.994739,-1.705279,-1.414272,0.576145,0.426281,3763.0,5135.0
4,-0.523105,1.137995,0.085889,-0.826413,-0.638095,1.481118,9820.0,21753.0
...,...,...,...,...,...,...,...,...
49995,-3.315216,1.137995,-1.705279,0.937163,-0.030975,-1.008297,780.0,32072.0
49996,-1.500344,1.137995,-0.809695,0.349304,-1.245215,-0.122234,2298.0,50748.0
49997,0.733344,-1.099672,-0.809695,-0.238555,1.790385,-1.050491,635.0,24051.0
49998,3.385849,0.242928,-2.600864,0.349304,-1.245215,0.468475,3633.0,4512.0


In [31]:
array2df_tail_3940 = array2df.tail(3940)
array2df_tail_3940

,depth_scaled,table_scaled,cut_scaled,color_scaled,clarity_scaled,carat_scaled,price_USD,index_org
50000,-0.243894,-0.652139,0.981473,-1.414272,2.397505,-0.987200,694.0,29200.0
50001,-0.034486,-0.652139,0.981473,1.525021,1.183265,-0.607459,2718.0,53703.0
50002,-0.941922,1.585529,0.085889,-0.238555,-1.245215,2.535955,17136.0,27011.0
50003,-0.523105,-0.204605,0.981473,0.937163,-0.030975,-0.565266,1727.0,46017.0
50004,0.454133,0.242928,-0.809695,-1.414272,-0.638095,-1.008297,388.0,41584.0
...,...,...,...,...,...,...,...,...
53935,0.942753,0.690462,-1.705279,0.937163,-0.638095,0.595055,5096.0,11825.0
53936,0.035317,-0.204605,0.981473,-0.238555,-1.245215,-0.164427,1866.0,47508.0
53937,2.129400,-0.652139,-2.600864,-0.238555,1.183265,1.481118,13553.0,25015.0
53938,-2.058766,0.690462,0.085889,1.525021,-1.245215,0.447378,4672.0,9818.0


In [42]:
#Store scaled input values back into SQL for input into machine learning

cursor = cnxn.cursor()
scaledip_insert = '''INSERT INTO diamonds_shuf69sclnzparm_50k (depth_scaled,table_scaled,cut_scaled,color_scaled,clarity_scaled,carat_scaled,price_USD,index_org)
               VALUES (?,?,?,?,?,?,?,?)
'''
cursor.fast_executemany = True
cursor.executemany(scaledip_insert, array2df_head_50k.values.tolist())
cnxn.commit()
cursor.close()
print("Input Scaled Values Done!")

#OK works.

Input Scaled Values Done!


In [44]:
array2df_tail_3940

,depth_scaled,table_scaled,cut_scaled,color_scaled,clarity_scaled,carat_scaled,price_USD,index_org
50000,-0.243894,-0.652139,0.981473,-1.414272,2.397505,-0.987200,694.0,29200.0
50001,-0.034486,-0.652139,0.981473,1.525021,1.183265,-0.607459,2718.0,53703.0
50002,-0.941922,1.585529,0.085889,-0.238555,-1.245215,2.535955,17136.0,27011.0
50003,-0.523105,-0.204605,0.981473,0.937163,-0.030975,-0.565266,1727.0,46017.0
50004,0.454133,0.242928,-0.809695,-1.414272,-0.638095,-1.008297,388.0,41584.0
...,...,...,...,...,...,...,...,...
53935,0.942753,0.690462,-1.705279,0.937163,-0.638095,0.595055,5096.0,11825.0
53936,0.035317,-0.204605,0.981473,-0.238555,-1.245215,-0.164427,1866.0,47508.0
53937,2.129400,-0.652139,-2.600864,-0.238555,1.183265,1.481118,13553.0,25015.0
53938,-2.058766,0.690462,0.085889,1.525021,-1.245215,0.447378,4672.0,9818.0


In [45]:
#Store scaled input values back into SQL for input into machine learning

cursor = cnxn.cursor()
scaledip_insert = '''INSERT INTO diamonds_shuf69sclnzparm_tail_3940 (depth_scaled,table_scaled,cut_scaled,color_scaled,clarity_scaled,carat_scaled,price_USD,index_org)
               VALUES (?,?,?,?,?,?,?,?)
'''
cursor.fast_executemany = True
cursor.executemany(scaledip_insert, array2df_tail_3940.values.tolist())
cnxn.commit()
cursor.close()
print("Input Scaled Values Done!")

#OK works.

Input Scaled Values Done!
